In [1]:
#Packages
import os
import pandas as pd
import itertools
import numpy as np
from operator import itemgetter

In [13]:
#Load Data
moving_average_shares = pd.read_csv('../temporary/moving_average_shares.csv', index_col='Unnamed: 0')
topics = pd.read_csv('../temporary/topics.csv')
moving_average_shares

,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,...,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890
1x2,0.000010,1.993856e-06,1.890401e-06,1.797580e-06,1.797580e-06,0.000025,0.000023,0.000023,0.000023,0.000023,...,0.002293,0.002321,0.002335,0.002333,0.002247,0.002226,0.002238,0.002245,0.002236,0.002221
1x3,0.000005,1.068301e-06,1.005507e-06,9.554093e-07,9.554093e-07,0.000012,0.000011,0.000011,0.000011,0.000011,...,0.000669,0.000691,0.000721,0.000737,0.000755,0.000758,0.000769,0.000787,0.000802,0.000786
1x4,0.000009,2.563470e-06,2.364779e-06,2.242189e-06,2.242189e-06,0.000020,0.000019,0.000019,0.000019,0.000019,...,0.000735,0.000739,0.000704,0.000693,0.000674,0.000659,0.000636,0.000626,0.000647,0.000630
1x5,0.000014,1.480231e-06,1.488265e-06,1.423555e-06,1.423555e-06,0.000212,0.000199,0.000199,0.000199,0.000199,...,0.001772,0.001853,0.001881,0.001907,0.001958,0.001996,0.002015,0.002068,0.002129,0.002141
1x6,0.000002,1.352189e-07,1.484902e-07,1.431995e-07,1.431995e-07,0.000005,0.000006,0.000006,0.000006,0.000006,...,0.000304,0.000328,0.000336,0.000359,0.000374,0.000373,0.000368,0.000363,0.000350,0.000349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57x59,0.000011,1.248851e-05,1.004193e-04,9.509673e-05,9.509673e-05,0.000084,0.000079,0.000079,0.000079,0.000079,...,0.000035,0.000035,0.000033,0.000033,0.000033,0.000033,0.000032,0.000033,0.000032,0.000033
57x60,0.000002,1.767700e-07,7.619845e-04,7.197240e-04,7.197240e-04,0.000632,0.000595,0.000595,0.000595,0.000595,...,0.000128,0.000133,0.000133,0.000142,0.000153,0.000153,0.000149,0.000148,0.000153,0.000155
58x59,0.003315,3.998958e-03,3.537325e-03,3.338580e-03,3.338580e-03,0.002930,0.002758,0.002758,0.002758,0.002758,...,0.000198,0.000202,0.000204,0.000199,0.000200,0.000204,0.000205,0.000209,0.000214,0.000214
58x60,0.000104,1.116548e-04,9.957842e-05,9.400082e-05,9.400082e-05,0.000086,0.000081,0.000081,0.000081,0.000081,...,0.000916,0.000920,0.000944,0.000950,0.000983,0.001013,0.001030,0.001046,0.001066,0.001057


In [3]:
topics

,1,2,topic_number
0,0.14322,paint pictur artist music engrav painter colou...,1
1,0.25957,town road church build built river stone wall ...,2
2,0.12400,franc pari french loui madam duke count napole...,3
3,0.19923,church christian christ bishop holi paul doctr...,4
4,0.36643,love heart beauti soul sweet dark night earth ...,5
5,0.05915,india chines china nativ indian bengal govern ...,6
6,0.15907,fig water iron engin pressur steam electr air ...,7
7,0.07679,acid solut heat carbon water sulphur iron gas ...,8
8,0.49381,exist refer period similar consist occur conne...,9
9,0.07330,vol lond fol folio calf copi pari par morocco ...,10


In [4]:
def category_score(topic, cat):
    c = set(cat) #turn category into set
    c.update([topic]) #add topic to set
    cross_combos = list(itertools.combinations(c,2)) #creates combinations of cross-topics
    cross_combos = ['x'.join(map(str,i)) for i in cross_combos] #easy referencing for 'moving_average_shares'
    print(cross_combos)
    return cross_combos

In [5]:
def category_shares(topics, shares, year, categories):
    #'categories' needs to be a dict with keys as category names and values as a list of category topics, i.e. 'Category': [1,2,3]
    tmp_dict = {}
    topic_numbers = list(topics['topic_number'])

    for category in categories:
        c = set
        c.update()

    df = pd.DataFrame.from_dict(tmp_dict)

## Playground

In [21]:
category = [33,34,47]
shares = moving_average_shares['1550']
topic_numbers = list(topics['topic_number'])
sets = {i:set([i] + category) for i in topic_numbers}
combos = {key:list(itertools.combinations(sorted(value),2)) for key,value in sets.items()}
cross_combos = {key:['x'.join(map(str,i)) for i in value] for key,value in combos.items()}
cross_shares = {key:[shares[str(i)] for i in value] for key,value in cross_combos.items()}
cross_sum = {key:sum(value) if key not in category else sum(value)*1.5 for key,value in cross_shares.items()}

cross_sum

{1: 4.391787821549061e-05,
 2: 3.743771177267266e-05,
 3: 9.851899614783309e-05,
 4: 0.00013682015307624963,
 5: 0.00016483520953064555,
 6: 3.6255084004967876e-05,
 7: 3.697154120960827e-05,
 8: 3.6801533091394e-05,
 9: 3.8290338058404235e-05,
 10: 5.054447682124767e-05,
 11: 4.094269097897618e-05,
 12: 0.0016667504057213726,
 13: 3.676327455179063e-05,
 14: 0.00017005202113510454,
 15: 0.0003169293079992292,
 16: 5.963133241447047e-05,
 17: 4.052908415416247e-05,
 18: 5.3186230061582315e-05,
 19: 3.918512474870575e-05,
 20: 5.5898118819865675e-05,
 21: 0.00031157944955847696,
 22: 7.806480799380354e-05,
 23: 6.904328846321804e-05,
 24: 3.767467290550988e-05,
 25: 8.809008822577574e-05,
 26: 3.8932417464796645e-05,
 27: 0.0006190071588781126,
 28: 3.623956914891243e-05,
 29: 3.861376170479744e-05,
 30: 3.633925968525193e-05,
 31: 6.224003884376612e-05,
 32: 3.978406557726135e-05,
 33: 5.4219311082583376e-05,
 34: 5.4219311082583376e-05,
 35: 3.670031549341529e-05,
 36: 7.7145000599730

In [18]:
s = {3,2,1}
print(sorted(s))

[1, 2, 3]


In [15]:
num = [1]

list(itertools.product(num, religion))

[(1, 2), (1, 3), (1, 4)]

In [ ]:
t =list(itemgetter(*ls)(moving_average_shares[1510]))
sum(t)

In [4]:
l1 = [1,2,3]
l2 = [4,5,6]
l3 = [7,8,9]

test = [l1,l2,l3]
for t in test:
    print(t)

[1, 2, 3]
[4, 5, 6]
[7, 8, 9]


In [11]:
mydict = {
    'one':l1,
    'two':l2,
    'three':l3}

for l in mydict:
    print(mydict[l])

df = pd.DataFrame.from_dict(mydict)
df

[1, 2, 3]
[4, 5, 6]
[7, 8, 9]


,one,two,three
0,1,4,7
1,2,5,8
2,3,6,9


In [16]:
moving_average_shares['1550']

1x2      6.046093e-08
1x3      2.046476e-07
1x4      4.012441e-06
1x5      4.252168e-07
1x6      7.824422e-09
             ...     
57x59    3.202851e-06
57x60    7.079243e-06
58x59    4.086480e-05
58x60    3.346370e-07
59x60    4.859878e-06
Name: 1550, Length: 1770, dtype: float64